# Identify CPU performance issues

https://docs.microsoft.com/en-us/azure/azure-sql/database/monitoring-with-dmvs  

If CPU consumption is above 80% for extended periods of time, consider the following troubleshooting steps:

## The CPU Issue is occurring now

If issue is occurring right now, there are two possible scenarios:

#### [](https://docs.microsoft.com/en-us/azure/azure-sql/database/monitoring-with-dmvs#many-individual-queries-that-cumulatively-consume-high-cpu)Many individual queries that cumulatively consume high CPU

Use the following query to identify top query hashes:

In [ ]:
/* This query outputs top 10 most used queries*/
SELECT TOP (10)
[database_name] = DB_NAME([qt].[dbid])
, [schema_name] = OBJECT_SCHEMA_NAME([qt].[objectid], [qt].[dbid])
, [object_name] = OBJECT_NAME([qt].[objectid], [qt].[dbid])
, [statement] = SUBSTRING([qt].[text], ([qs].[statement_start_offset] / 2) + 1, ((CASE [qs].[statement_end_offset] WHEN -1 THEN DATALENGTH([qt].[text]) ELSE [qs].[statement_end_offset] END - [qs].[statement_start_offset]) / 2) + 1)
, [qs].[execution_count]
, [qs].[total_logical_reads]
, [qs].[last_logical_reads]
, [qs].[total_logical_writes]
, [qs].[last_logical_writes]
, [qs].[total_worker_time]
, [qs].[last_worker_time]
, [total_elapsed_time_in_S] = [qs].[total_elapsed_time] / 1000000
, [last_elapsed_time_in_S] = [qs].[last_elapsed_time] / 1000000
, [qs].[last_execution_time]
, [qp].[query_plan]
FROM [sys].[dm_exec_query_stats] AS [qs]
CROSS APPLY [sys].[dm_exec_sql_text]([qs].[sql_handle]) AS [qt]
CROSS APPLY [sys].[dm_exec_query_plan]([qs].[plan_handle]) AS [qp]
ORDER BY [qs].[total_logical_reads] DESC

In [ ]:
PRINT '-- top 10 Active CPU Consuming Queries (aggregated)--';
SELECT TOP 10 GETDATE() runtime, *
FROM (SELECT query_stats.query_hash, SUM(query_stats.cpu_time) 'Total_Request_Cpu_Time_Ms', SUM(logical_reads) 'Total_Request_Logical_Reads', MIN(start_time) 'Earliest_Request_start_Time', COUNT(*) 'Number_Of_Requests', SUBSTRING(REPLACE(REPLACE(MIN(query_stats.statement_text), CHAR(10), ' '), CHAR(13), ' '), 1, 256) AS "Statement_Text"
    FROM (SELECT req.*, SUBSTRING(ST.text, (req.statement_start_offset / 2)+1, ((CASE statement_end_offset WHEN -1 THEN DATALENGTH(ST.text)ELSE req.statement_end_offset END-req.statement_start_offset)/ 2)+1) AS statement_text
          FROM sys.dm_exec_requests AS req
                CROSS APPLY sys.dm_exec_sql_text(req.sql_handle) AS ST ) AS query_stats
    GROUP BY query_hash) AS t
ORDER BY Total_Request_Cpu_Time_Ms DESC;

In [ ]:
/* Gives us the top 10 queries that are consuming the most IO */	
/* https://blog.sqlauthority.com/2014/07/29/sql-server-ssms-top-queries-by-cpu-and-io/ */	
SELECT TOP 10
	creation_time
	,last_execution_time
	,total_logical_reads AS [LogicalReads] , total_logical_writes AS [LogicalWrites] 
	,execution_count
	,total_logical_reads+total_logical_writes AS [AggIO] 
	,(total_logical_reads+total_logical_writes)/(execution_count+0.0) AS [AvgIO]
	,st.TEXT
FROM sys.dm_exec_query_stats qs
CROSS APPLY sys.dm_exec_sql_text(sql_handle) st
WHERE total_logical_reads+total_logical_writes > 0
AND sql_handle IS NOT NULL
ORDER BY [AggIO] DESC;

## Long running queries that consume CPU are still running

Use the following query to identify these queries:

In [ ]:
PRINT '--top 10 Active CPU Consuming Queries by sessions--';
SELECT TOP 10 req.session_id, req.start_time, cpu_time 'cpu_time_ms', OBJECT_NAME(ST.objectid, ST.dbid) 'ObjectName', SUBSTRING(REPLACE(REPLACE(SUBSTRING(ST.text, (req.statement_start_offset / 2)+1, ((CASE statement_end_offset WHEN -1 THEN DATALENGTH(ST.text)ELSE req.statement_end_offset END-req.statement_start_offset)/ 2)+1), CHAR(10), ' '), CHAR(13), ' '), 1, 512) AS statement_text
FROM sys.dm_exec_requests AS req
    CROSS APPLY sys.dm_exec_sql_text(req.sql_handle) AS ST
ORDER BY cpu_time DESC;
GO

# Identify IO performance issues

<span style="font-size: 16px; letter-spacing: 0.15px;">IO Usage is split into two types: Data IO and Log IO. A&nbsp;a combination of the two can also cause issues with IO performance. To get current IO usage, run the following queries on your database:</span>  

When identifying IO performance issues, the top wait types associated with IO issues are:

- `PAGEIOLATCH_*`
    
    For data file IO issues (including `PAGEIOLATCH_SH`, `PAGEIOLATCH_EX`, `PAGEIOLATCH_UP`). If the wait type name has IO in it, it points to an IO issue. If there is no IO in the page latch wait name, it points to a different type of problem (for example, tempdb contention).
    
- `WRITE_LOG`
    
    For transaction log IO issues.
    

### [](https://docs.microsoft.com/en-us/azure/azure-sql/database/monitoring-with-dmvs#if-the-io-issue-is-occurring-right-now)If the IO issue is occurring right now

Use the [sys.dm\_exec\_requests](https://docs.microsoft.com/en-us/sql/relational-databases/system-dynamic-management-views/sys-dm-exec-requests-transact-sql) or [sys.dm\_os\_waiting\_tasks](https://docs.microsoft.com/en-us/sql/relational-databases/system-dynamic-management-views/sys-dm-os-waiting-tasks-transact-sql) to see the `wait_type` and `wait_time`.

#### [](https://docs.microsoft.com/en-us/azure/azure-sql/database/monitoring-with-dmvs#identify-data-and-log-io-usage)Identify data and log IO usage

Use the following query to identify data and log IO usage. If the data or log IO is above 80%, it means users have used the available IO for the SQL Database service tier.

In [ ]:
--To get current IO usage, run the following query on your database:
SELECT end_time, avg_data_io_percent, avg_log_write_percent
FROM sys.dm_db_resource_stats
ORDER BY end_time DESC

In [ ]:
SELECT io_stall_write_ms/num_of_writes as avg_tlog_io_write_ms, * 
FROM sys.dm_io_virtual_file_stats
(db_id('Adventureworks'), 2);
GO

In [ ]:
/* Gives us the top 10 queries that are consuming the most IO */
/* https://blog.sqlauthority.com/2014/07/29/sql-server-ssms-top-queries-by-cpu-and-io/ */	
SELECT TOP 10
	creation_time
	,last_execution_time
	,total_logical_reads AS [LogicalReads] , total_logical_writes AS [LogicalWrites] 
	,execution_count
	,total_logical_reads+total_logical_writes AS [AggIO] 
	,(total_logical_reads+total_logical_writes)/(execution_count+0.0) AS [AvgIO]
	,st.TEXT
FROM sys.dm_exec_query_stats qs
CROSS APPLY sys.dm_exec_sql_text(sql_handle) st
WHERE total_logical_reads+total_logical_writes > 0
AND sql_handle IS NOT NULL
ORDER BY [AggIO] DESC;